<a href="https://colab.research.google.com/github/kittisaktaoma/colab_practice/blob/master/CalcSynergy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import Library 



In [ ]:
!wget -c https://repo.continuum.io/miniconda/Miniconda3-py37_4.8.3-Linux-x86_64.sh
!chmod +x Miniconda3-py37_4.8.3-Linux-x86_64.sh
!time bash ./Miniconda3-py37_4.8.3-Linux-x86_64.sh -b -f -p /usr/local
!time conda install -q -y -c conda-forge rdkit
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

In [ ]:
! pip install keras
! pip install tensorflow
! pip install keras-gcn
! pip install numpy
! pip install matplotlib
! pip install scipy
! pip install deepchem
! pip install keras-tuner
! pip install chembl_webresource_client

In [8]:
#import deepchem as dc
import tensorflow as tf
import tensorflow.keras.layers as layers
import pandas as pd
import numpy as np
#import deepchem as dc
from rdkit import Chem
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import BayesianOptimization
from chembl_webresource_client.new_client import new_client

#Boolean GRN

#Target Protein

## STITCH

In [64]:
df = pd.read_csv("https://raw.githubusercontent.com/kittisaktaoma/colab_practice/master/sample.txt",sep = "\t")
df_list_drug = pd.concat([df["drug_row"], df["drug_col"]]).unique().tolist()

In [ ]:
def target_protein_stith(x):
  print(x)
  url = "http://stitch.embl.de/api/tsv/interactors?identifier="+x+"&species=9606"
  res = pd.read_csv(url,delimiter=".")
  res['Drug_name'] = x
  res_02 = res.iloc[1:,]
  return (res_02)

def get_result_stitch(x):
  try:
    return (target_protein_stith(x))
  except:
    pass
 
#res_target_protein_stitch = pd.concat(map(get_result_stitch,df_list_drug))
res_target_protein_stitch

## Chembl

In [ ]:
! pip install chembl_webresource_client

In [78]:
from chembl_webresource_client.new_client import new_client

In [87]:
def get_protein_chembl (x):
  print(x)
  activity = new_client.mechanism
  mechanism_records = new_client.mechanism.filter(molecule_chembl_id=x)
  res = pd.DataFrame.from_dict(mechanism_records)
  return (res)

def get_result_chembl(x):
  try:
    return (get_protein_chembl(x))
  except:
    pass
 
#res_target_protein_stitch = pd.concat(map(get_result_chembl,df_list_drug[:2]))
#res_target_protein_stitch

molecule = new_client.molecule
m1 = molecule.get('5-FU')
m1

RetryError: ignored

#Drug Structure (GCN layer)

In [ ]:
def tune_nn_model(hp):
    model = keras.Sequential()
    
    drop_rate = hp.Choice('drop_rate',
                            [
                              0.0,
                              0.1,
                              0.2,
                              0.3,
                              0.4,
                              0.5,
                              0.6,
                              0.7,
                              0.8,
                              0.9
                            ])

     
    model.add(keras.layers.Dense(
      units=28,
      activation="relu",
      input_shape=[df_x.shape[1]]
    ))
    model.add(keras.layers.Dropout(rate=drop_rate))
    model.add(keras.layers.Dense(16, activation='relu'))
    model.add(keras.layers.Dropout(rate=drop_rate))
    model.add(keras.layers.Dense(1, activation='sigmoid')) 
    lr = hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])
    momentum = hp.Choice('momentum', [0.0, 0.2, 0.4, 0.6, 0.8, 0.9])

    model.compile(
        optimizer=keras.optimizers.SGD(lr, momentum=momentum),
        loss = 'binary_crossentropy',
        metrics = ['accuracy'])
    return model

In [ ]:
b_tuner = BayesianOptimization(
    tune_nn_model,
    objective='accuracy',
    max_trials=20,
    executions_per_trial=5,
    directory='test_dir_03',
    project_name='b_tune_nn'
)

INFO:tensorflow:Reloading Oracle from existing project test_dir_03/b_tune_nn/oracle.json


In [ ]:
df_x_x = df_x.to_numpy()
df_y_y = df_y.to_numpy()

In [ ]:
b_tuner.search(df_x_x,df_y_y)

In [ ]:
b_tuner.results_summary()

In [ ]:
# Retrieve the best model and display its architecture
best_model = b_tuner.get_best_models(num_models=1)[0]
best_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 28)                112       
_________________________________________________________________
dropout (Dropout)            (None, 28)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                464       
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 17        
Total params: 593
Trainable params: 593
Non-trainable params: 0
_________________________________________________________________
